In [ ]:
# pip install surprise

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#for collaborative filtering
import os
import math
import random

from surprise import accuracy, Reader, Dataset, dump
from surprise import NormalPredictor, KNNBasic, SVD, SVDpp
from surprise.model_selection import cross_validate, GridSearchCV

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
Ratings = pd.read_csv('Ratings.csv')
Books = pd.read_csv('DataGabungan.csv')

In [ ]:
Ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [ ]:
Books

,Unnamed: 0,ISBN,user_id,rating,title,author,publisher,image,year,average_rating,count_ratings
0,5,0060283130,7,56,The Reptile Room (A Series of Unfortunate Even...,Lemony Snicket,HarperCollins,http://images.amazon.com/images/P/0060283130.0...,1999,8.00,49
1,6,0060392452,58,484,Stupid White Men ...and Other Sorry Excuses fo...,Michael Moore,Regan Books,http://images.amazon.com/images/P/0060392452.0...,2002,8.34,3364
2,7,0060502258,58,453,The Divine Secrets of the Ya-Ya Sisterhood: A ...,Rebecca Wells,HarperTorch,http://images.amazon.com/images/P/0060502258.0...,2002,7.81,3364
3,11,0060809833,10,91,Brave New World,Aldous Huxley,Harpercollins,http://images.amazon.com/images/P/0060809833.0...,1989,9.10,100
4,12,0060830956,8,70,Brave New World,Aldous Huxley,HarperCollins Publishers,http://images.amazon.com/images/P/0060830956.0...,1984,8.75,64
...,...,...,...,...,...,...,...,...,...,...,...
358,619,1573226122,8,63,Lord of the Flies,William Golding,Riverhead Books,http://images.amazon.com/images/P/1573226122.0...,1997,7.88,64
359,620,1573228214,16,119,High Fidelity,Nick Hornby,Riverhead Books,http://images.amazon.com/images/P/1573228214.0...,2000,7.44,256
360,621,1573229326,33,232,How to Be Good,Nick Hornby,Riverhead Books,http://images.amazon.com/images/P/1573229326.0...,2002,7.03,1089
361,628,185326041X,6,49,The Great Gatsby,F. Scott Fitzgerald,Lb May &amp; Assoc Inc,http://images.amazon.com/images/P/185326041X.0...,1999,8.17,36


In [ ]:
# #updating column names in ratings dataset
# Ratings.rename(columns = {"User-ID":"user_id", "Book-Rating": "rating"}, inplace=True)

# Ratings = Ratings[Ratings['rating'] != 0]

# counters = Ratings.groupby(['user_id', 'ISBN']).size().reset_index(name='rating_count')

# # Filter users who have rated more than 30 books and books that have more than 40 ratings
# user_filter = counters.groupby('user_id').size() >= 30
# book_filter = counters.groupby('ISBN').size() >= 40

# # Apply the filters to the original dataset
# Ratings = Ratings[Ratings['user_id'].isin(user_filter[user_filter].index) & Ratings['ISBN'].isin(book_filter[book_filter].index)]

# Books.drop(['Unnamed: 0', 'user_id', 'rating'], axis=1, inplace=True)

In [ ]:


# Step 1: Count the number of unique users who rated each book
counters = Ratings.groupby('ISBN')['user_id'].nunique().reset_index(name='user_count')

# Step 2: Find books that have been rated by at least 40 users
book_filter = counters[counters['user_count'] >= 40]['ISBN']

# Step 3: Find users who have rated at least 30 books
user_filter = Ratings.groupby('user_id').size() >= 30

# Step 4: Apply the filters to the original dataset
Ratings = Ratings[Ratings['user_id'].isin(user_filter[user_filter].index) & Ratings['ISBN'].isin(book_filter)]

Books.drop(['Unnamed: 0', 'user_id', 'rating'], axis=1, inplace=True)

KeyError: 'Column not found: user_id'

In [ ]:
Ratings

In [ ]:
Books

In [ ]:
Ratings.rating.plot.box()

In [ ]:
Books

**Collaborative Filtering**



In [ ]:
#to have reproducible experiments
my_seed = 0
random.seed(my_seed)
np.random.seed(my_seed)

#Load the full dataset
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(Ratings, reader)

#shuffle the ratings for unbiased result
all_ratings = data.raw_ratings
random.shuffle(all_ratings)

#split data into train and test data with the ratio 70:30
threshold = int(0.7 * len(all_ratings))
train_ratings = all_ratings[:threshold]
test_ratings = all_ratings[threshold:]

def book_read(user_id):
    '''Take user_id and return list of book that user has read'''
    books_list = list(Books['ISBN'])
    book_read_list = list(Ratings['ISBN'][Ratings['user_id'] == user_id])
    return books_list, book_read_list

In [ ]:
# Menghitung jumlah rating per user
user_rating_counts = Ratings.groupby('user_id').size()

# Memfilter pengguna dengan lebih dari 30 rating
# filtered_users = user_rating_counts[user_rating_counts >= 30]

# Mengurutkan hasil berdasarkan count_rating secara descending
filtered_users_sorted = user_rating_counts.sort_values(ascending=False)

# Menampilkan hasil dalam DataFrame dengan format yang diinginkan
df_filtered_users = pd.DataFrame({'user_id': filtered_users_sorted.index, 'count_ratings': filtered_users_sorted.values})
df_filtered_users

a. Normal Predictor

In [ ]:
%%time

# prepare train data
data.raw_ratings = train_ratings

#select algorithm
npred = NormalPredictor()

# cross validation for train data
np_result = cross_validate(npred, data, measures=['RMSE'], cv=5, verbose=True, n_jobs=1)

#retrain whole train test
trainset = data.build_full_trainset()
npred.fit(trainset)
print('\n')

# Compute RMSE on trainset (without fold)
np_train_pred = npred.test(trainset.build_testset())
print('Train RMSE:')
train_rmse = accuracy.rmse(np_train_pred)
print('Train MAE:')
train_mae = accuracy.mae(np_train_pred)
print('\n')

#compute RMSE on testset
testset = data.construct_testset(test_ratings)
np_test_pred = npred.test(testset)
print('Test RMSE:')
test_rmse = accuracy.rmse(np_test_pred)
print('Test MAE:')
test_mae = accuracy.mae(np_test_pred)

data.raw_ratings = all_ratings
npred.fit(data.build_full_trainset())

def get_recommendation_npred(user_id, n=5):
    '''Give n recommendation to user_id'''

    all_books, user_books =  book_read(user_id)
    next_books = [book for book in all_books if book not in user_books]

    if n <= len(next_books):
        ratings = []
        for book in next_books:
            est = npred.predict(user_id, book).est
            ratings.append((book, est))
        ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
        book_ids = [id for id, rate in ratings[:n]]
        return Books[Books.ISBN.isin(book_ids)][['ISBN', 'title', 'author', 'average_rating', 'count_ratings']]
    else:
        print('Please reduce your recommendation request')
        return

In [ ]:
# uncomment this code to dump the calculation result for future use
dump.dump('./dump_np', np_test_pred, npred)
dump

In [ ]:
get_recommendation_npred(11676, 5)

K-Nearest Neighbour

In [ ]:
%%time
# change data to trainset
data.raw_ratings = train_ratings

# select algorithm
sim_options = {"name": "cosine",
              "user_based": True}
knn = KNNBasic(sim_options=sim_options)

# cross validation for train data
knn_result = cross_validate(knn, data, measures=['RMSE'], cv=5, verbose=True, n_jobs = 1)

# retrain whole train test
trainset = data.build_full_trainset()
knn.fit(trainset)

# Compute RMSE on trainset (without fold)
knn_train_pred = knn.test(trainset.build_testset())
print('Train RMSE:')
train_rmse = accuracy.rmse(knn_train_pred)
print('Train MAE:')
train_mae = accuracy.mae(knn_train_pred)
print('\n')

# compute RMSE on testset
testset = data.construct_testset(test_ratings)
knn_test_pred = knn.test(testset)
print('Test RMSE:')
test_rmse = accuracy.rmse(knn_test_pred)
print('Test MAE:')
test_mae = accuracy.mae(knn_test_pred)
print('\n')

data.raw_ratings = all_ratings
knn.fit(data.build_full_trainset())

def get_recommendation_knn(user_id, n=5):
    '''Give n recommendation to user_id'''

    all_books, user_books =  book_read(user_id)
    next_books = [book for book in all_books if book not in user_books]

    if n <= len(next_books):
        ratings = []
        for book in next_books:
            est = knn.predict(user_id, book).est
            ratings.append((book, est))
        ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
        book_ids = [id for id, rate in ratings[:n]]
        return Books[Books.ISBN.isin(book_ids)][['ISBN', 'title', 'author', 'average_rating', 'count_ratings']]
    else:
        print('Please reduce your recommendation request')
        return

In [ ]:
dump.dump('./dump_knn', knn_test_pred, knn)

In [ ]:
def get_recommendation_knn(user_id, n=5):
    '''Give n recommendation to user_id'''

    all_books, user_books =  book_read(user_id)
    next_books = [book for book in all_books if book not in user_books]

    if n <= len(next_books):
        ratings = []
        for book in next_books:
            est = knn.predict(user_id, book).est
            ratings.append((book, est))
        ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
        book_ids = [id for id, rate in ratings[:n]]
        return Books[Books.ISBN.isin(book_ids)][['ISBN', 'title', 'author', 'average_rating', 'count_ratings']]
    else:
        print('Please reduce your recommendation request')
        return

In [ ]:
get_recommendation_knn(11676, 5)

SVD (Singular Value Decomposition)

In [ ]:
%%time
#change data to trainset
data.raw_ratings = train_ratings

#select algorithm
svd = SVD(random_state=0)

#retrain whole train test
trainset = data.build_full_trainset()
svd.fit(trainset)

# Compute RMSE on trainset (without fold)
svd_train_pred = svd.test(trainset.build_testset())
print('Train RMSE:')
train_rmse = accuracy.rmse(svd_train_pred)
print('Train MAE:')
train_mae = accuracy.mae(svd_train_pred)
print('\n')

#compute RMSE on testset
testset = data.construct_testset(test_ratings)
svd_test_pred = svd.test(testset)
print('Test RMSE:')
test_rmse = accuracy.rmse(svd_test_pred)
print('Test MAE:')
train_mae = accuracy.mae(svd_test_pred)
print('\n')

data.raw_ratings = all_ratings
svd.fit(data.build_full_trainset())

def get_recommendation_svd(user_id, n=5):
    '''Give n recommendation to user_id'''

    all_books, user_books =  book_read(user_id)
    next_books = [book for book in all_books if book not in user_books]

    if n <= len(next_books):
        ratings = []
        for book in next_books:
            est = svd.predict(user_id, book).est
            ratings.append((book, est))
        ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
        book_ids = [id for id, rate in ratings[:n]]
        return Books[Books.ISBN.isin(book_ids)][['ISBN', 'title', 'author', 'average_rating', 'count_ratings']]
    else:
        print('Please reduce your recommendation request')
        return

In [ ]:
# uncomment this code to dump the calculation result for future use
dump.dump('./dump_svd', svd_test_pred, svd)
dump

In [ ]:
get_recommendation_svd(11676, 5)

SVD++


In [ ]:
%%time
#change data to trainset
data.raw_ratings = train_ratings

#select algorithm
svdpp = SVDpp(random_state=0)

#retrain whole train test
trainset = data.build_full_trainset()
svdpp.fit(trainset)

# Compute RMSE on trainset (without fold)
svdpp_train_pred = svdpp.test(trainset.build_testset())
print('Train RMSE:')
train_rmse = accuracy.rmse(svdpp_train_pred)
print('Train MAE:')
train_mae = accuracy.mae(svdpp_train_pred)
print('\n')


#compute RMSE on testset
testset = data.construct_testset(test_ratings)
svdpp_test_pred = svdpp.test(testset)
print('Test RMSE:')
test_rmse = accuracy.rmse(svdpp_test_pred)
print('Test MAE:')
train_mae = accuracy.mae(svdpp_test_pred)
print('\n')


data.raw_ratings = all_ratings
svdpp.fit(data.build_full_trainset())

def get_recommendation_svdpp(user_id, n=5):
    '''Give n recommendation to user_id'''

    all_books, user_books =  book_read(user_id)
    next_books = [book for book in all_books if book not in user_books]

    if n <= len(next_books):
        ratings = []
        for book in next_books:
            est = svdpp.predict(user_id, book).est
            ratings.append((book, est))
        ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
        book_ids = [id for id, rate in ratings[:n]]
        return Books[Books.ISBN.isin(book_ids)][['ISBN', 'title', 'author', 'average_rating', 'count_ratings']]
    else:
        print('Please reduce your recommendation request')
        return

In [ ]:
data.build_full_trainset()

In [ ]:
# uncomment this code to dump the calculation result for future use
dump.dump('./dump_svdpp', svdpp_test_pred, svdpp)
dump

In [ ]:
get_recommendation_svdpp(11676, 5)

**Precision, Recall, F1 Score**

K = 5 | Threshold = 4.0

In [ ]:
import numpy as np
from collections import defaultdict

# Function to get top K recommendations for each user
def get_top_k_recommendations(predictions, k):
    # Map the predictions to each user.
    top_k_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_k_recs[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_k_recs.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_k_recs[uid] = user_ratings[:k]

    return top_k_recs

# Function to calculate Precision@K, Recall@K, and F1@K
def precision_recall_f1_at_k(predictions, k, threshold=4.0):
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    f1_scores = dict()

    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # Get the top K items
        user_ratings_top_k = user_ratings[:k]

        # Count relevant items in top K
        n_relevant = sum((true_r >= threshold) for (_, true_r) in user_ratings_top_k)

        # Count relevant items in all items rated by user
        n_relevant_total = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Precision@K
        precisions[uid] = n_relevant / k

        # Recall@K
        recalls[uid] = n_relevant / n_relevant_total if n_relevant_total != 0 else 0

        # F1@K
        if precisions[uid] + recalls[uid] > 0:
            f1_scores[uid] = 2 * (precisions[uid] * recalls[uid]) / (precisions[uid] + recalls[uid])
        else:
            f1_scores[uid] = 0

    # Average precision and recall over all users
    mean_precision = np.mean(list(precisions.values()))
    mean_recall = np.mean(list(recalls.values()))
    mean_f1 = np.mean(list(f1_scores.values()))

    return mean_precision, mean_recall, mean_f1

# Example usage with predictions from your model
top_k = 5
knn_top_k_recommendations = get_top_k_recommendations(knn_test_pred, top_k)
knn_mean_precision, knn_mean_recall, knn_mean_f1 = precision_recall_f1_at_k(knn_test_pred, top_k)

print(f'KNN Precision@{top_k}: {knn_mean_precision:.4f}')
print(f'KNN Recall@{top_k}: {knn_mean_recall:.4f}')
print(f'KNN F1@{top_k}: {knn_mean_f1:.4f}')
print('\n')

np_top_k_recommendations = get_top_k_recommendations(np_test_pred, top_k)
np_mean_precision, np_mean_recall, np_mean_f1 = precision_recall_f1_at_k(np_test_pred, top_k)

print(f'Normal Predictor Precision@{top_k}: {np_mean_precision:.4f}')
print(f'Normal Predictor Recall@{top_k}: {np_mean_recall:.4f}')
print(f'Normal Predictor F1@{top_k}: {np_mean_f1:.4f}')
print('\n')

svd_top_k_recommendations = get_top_k_recommendations(svd_test_pred, top_k)
svd_mean_precision, svd_mean_recall, svd_mean_f1 = precision_recall_f1_at_k(svd_test_pred, top_k)

print(f'SVD Precision@{top_k}: {svd_mean_precision:.4f}')
print(f'SVD Recall@{top_k}: {svd_mean_recall:.4f}')
print(f'SVD F1@{top_k}: {svd_mean_f1:.4f}')
print('\n')

svdpp_top_k_recommendations = get_top_k_recommendations(svdpp_test_pred, top_k)
svdpp_mean_precision, svdpp_mean_recall, svdpp_mean_f1 = precision_recall_f1_at_k(svdpp_test_pred, top_k)

print(f'SVD++ Precision@{top_k}: {svdpp_mean_precision:.4f}')
print(f'SVD++ Recall@{top_k}: {svdpp_mean_recall:.4f}')
print(f'SVD++ F1@{top_k}: {svdpp_mean_f1:.4f}')

In [ ]:
import numpy as np
from collections import defaultdict

# Function to get top K recommendations for each user
def get_top_k_recommendations(predictions, k):
    # Map the predictions to each user.
    top_k_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_k_recs[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_k_recs.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_k_recs[uid] = user_ratings[:k]

    return top_k_recs

# Function to calculate Precision@K, Recall@K, and F1@K
def precision_recall_f1_at_k(predictions, k, threshold=4.0):
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    f1_scores = dict()

    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # Get the top K items
        user_ratings_top_k = user_ratings[:k]

        # Count relevant items in top K
        n_relevant = sum((true_r >= threshold) for (_, true_r) in user_ratings_top_k)

        # Count relevant items in all items rated by user
        n_relevant_total = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Precision@K
        precisions[uid] = (n_relevant / k) * 100

        # Recall@K
        recalls[uid] = (n_relevant / n_relevant_total) * 100 if n_relevant_total != 0 else 0

        # F1@K
        if precisions[uid] + recalls[uid] > 0:
            f1_scores[uid] = 2 * (precisions[uid] * recalls[uid]) / (precisions[uid] + recalls[uid])
        else:
            f1_scores[uid] = 0

    # Average precision and recall over all users
    mean_precision = np.mean(list(precisions.values()))
    mean_recall = np.mean(list(recalls.values()))
    mean_f1 = np.mean(list(f1_scores.values()))

    return mean_precision, mean_recall, mean_f1

# Example usage with predictions from your model
top_k = 5
knn_top_k_recommendations = get_top_k_recommendations(knn_test_pred, top_k)
knn_mean_precision, knn_mean_recall, knn_mean_f1 = precision_recall_f1_at_k(knn_test_pred, top_k)

print(f'KNN Precision@{top_k}: {knn_mean_precision:.2f}%')
print(f'KNN Recall@{top_k}: {knn_mean_recall:.2f}%')
print(f'KNN F1@{top_k}: {knn_mean_f1:.2f}%')
print('\n')

np_top_k_recommendations = get_top_k_recommendations(np_test_pred, top_k)
np_mean_precision, np_mean_recall, np_mean_f1 = precision_recall_f1_at_k(np_test_pred, top_k)

print(f'Normal Predictor Precision@{top_k}: {np_mean_precision:.2f}%')
print(f'Normal Predictor Recall@{top_k}: {np_mean_recall:.2f}%')
print(f'Normal Predictor F1@{top_k}: {np_mean_f1:.2f}%')
print('\n')

svd_top_k_recommendations = get_top_k_recommendations(svd_test_pred, top_k)
svd_mean_precision, svd_mean_recall, svd_mean_f1 = precision_recall_f1_at_k(svd_test_pred, top_k)

print(f'SVD Precision@{top_k}: {svd_mean_precision:.2f}%')
print(f'SVD Recall@{top_k}: {svd_mean_recall:.2f}%')
print(f'SVD F1@{top_k}: {svd_mean_f1:.2f}%')
print('\n')

svdpp_top_k_recommendations = get_top_k_recommendations(svdpp_test_pred, top_k)
svdpp_mean_precision, svdpp_mean_recall, svdpp_mean_f1 = precision_recall_f1_at_k(svdpp_test_pred, top_k)

print(f'SVD++ Precision@{top_k}: {svdpp_mean_precision:.2f}%')
print(f'SVD++ Recall@{top_k}: {svdpp_mean_recall:.2f}%')
print(f'SVD++ F1@{top_k}: {svdpp_mean_f1:.2f}%')


Evaluation

In [ ]:
#uncomment this code to load the dump file
np_test_pred, npred = dump.load('./dump_np')
knn_test_pred, knn = dump.load('./dump_knn')
svd_test_pred, svd = dump.load('./dump_svd')
svdpp_test_pred, svdpp = dump.load('./dump_svdpp')

In [ ]:
#making the calculation result into dataframe
DataFrame_np = pd.DataFrame(np_test_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])
DataFrame_knn = pd.DataFrame(knn_test_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])
DataFrame_svd = pd.DataFrame(svd_test_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])
DataFrame_svdpp = pd.DataFrame(svdpp_test_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])

In [ ]:
import sys

memory_usage = sys.getsizeof(get_recommendation_svdpp)
print("Memory usage of get_recommendation_svdpp:", memory_usage, "bytes")

In [ ]:
import sys

memory_usage = sys.getsizeof(get_recommendation_svd)
print("Memory usage of my_list:", memory_usage, "bytes")

In [ ]:
import sys

memory_usage = sys.getsizeof(get_recommendation_knn)
print("Memory usage of get_recommendation_svdpp:", memory_usage, "bytes")

In [ ]:
import sys

memory_usage = sys.getsizeof(get_recommendation_npred)
print("Memory usage of my_list:", memory_usage, "bytes")

In [ ]:
data = [['NP', 4.2187, '18,45s', 144], ['SVD', 3.5074, '67,7s', 144],  ['SVD++', 3.5622, '2h 8m 16s', 144]]
df = pd.DataFrame(data, columns=['Model', 'RMSE', 'Duration', 'Memory Use (Byte)'])
df

Comparing Prediction Distribution

In [ ]:
# Mengembalikan nilai prediksi ke skala asli
DataFrame_np['est'] = DataFrame_np['est']
DataFrame_knn['est'] = DataFrame_knn['est']
DataFrame_svd['est'] = DataFrame_svd['est']
DataFrame_svdpp['est'] = DataFrame_svdpp['est']

# Membuat plot histogram untuk distribusi prediksi
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15, 6), sharey=True, constrained_layout=True)
DataFrame_np['est'].plot(kind='hist', title='NP', ax=ax1, color='#45818E')
DataFrame_knn['est'].plot(kind='hist', title='KNN', ax=ax2, color='#45818E')
DataFrame_svd['est'].plot(kind='hist', title='SVD', ax=ax3, color='#92BA92')
DataFrame_svdpp['est'].plot(kind='hist', title='SVDpp', ax=ax4, color='#F3936B')

# Menyesuaikan nilai pada sumbu y
# current_values = plt.gca().get_yticks()
# plt.gca().set_yticklabels(['{:,.0f}K'.format(x) for x in current_values])
# plt.suptitle('Prediction Distribution in Each Model', weight='bold', fontsize=14)
# plt.show()

current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}K'.format(x) for x in current_values])
plt.suptitle('Prediction Distribution in Each Model', weight='bold', fontsize=14)
plt.show()

In [ ]:
# # ngeprediksi setiap count rating yang diberikan
# DataFrame_np['est'] = DataFrame_np['est'] * 2
# DataFrame_knn['est'] = DataFrame_knn['est'] * 2
# DataFrame_svd['est'] = DataFrame_svd['est'] * 2
# DataFrame_svdpp['est'] = DataFrame_svdpp['est'] * 2

# #membuat plot
# figure, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4,  figsize=(15, 6), sharey=True, constrained_layout=True)
# DataFrame_np['est'].plot(kind='hist', title='NP', ax=ax1, color='#45818E')
# DataFrame_knn['est'].plot(kind='hist', title='KNN', ax=ax2, color='#45818E')
# DataFrame_svd['est'].plot(kind='hist', title='SVD', ax=ax3, color='#92BA92')
# DataFrame_svdpp['est'].plot(kind='hist', title='SVDpp', ax=ax4, color='#F3936B')

# current_values = plt.gca().get_yticks() / 1000
# plt.gca().set_yticklabels(['{:,.0f}'.format(x) + 'K' for x in current_values])
# plt.suptitle('Prediction Distribution in Each Model', weight='bold', fontsize=14)
# plt.show()

Comparison of rating estimation in each model

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
sns.kdeplot(DataFrame_np['est'], shade=False, color='#45818E', label = 'NP', ax=ax)
sns.kdeplot(DataFrame_knn['est'], shade=False, color='#45818E', label = 'KNN', ax=ax)
sns.kdeplot(DataFrame_svd['est'], shade=False, color='#92BA92', label = 'SVD', ax=ax)
sns.kdeplot(DataFrame_svdpp['est'], shade=False, color='#F3936B', label = 'SVDpp', ax=ax)
ax.legend()
plt.xlabel('Rating prediction')
plt.ylabel('Density')
plt.title('Comparison of rating estimation in each model', weight='bold', fontsize=14)
plt.show()

Hyperparameter Tuning On Best Model

In [ ]:
#Load the full dataset
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(Ratings, reader)

In [ ]:
# prepare train data
data.raw_ratings = train_ratings

In [ ]:
%%time

grid = {'n_epochs': [20, 30],
        'lr_all': [.005, .001],
        'reg_all': [0.02, 0.04]}

gs = GridSearchCV(SVD, grid, measures=['RMSE'], cv=2, n_jobs=1)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

In [ ]:
#select best algorithm
svdtuned = gs.best_estimator['rmse']

In [ ]:
%%time

#retrain whole train test (without fold)
trainset = data.build_full_trainset()
svdtuned.fit(trainset)

In [ ]:
%%time

# Compute RMSE on trainset (without fold)
svdtuned_train_pred = svdtuned.test(trainset.build_testset())
print('Train RMSE:')
train_rmse = accuracy.rmse(svdtuned_train_pred)

In [ ]:
%%time

#compute RMSE on testset
testset = data.construct_testset(test_ratings)
svdtuned_test_pred = svdtuned.test(testset)
print('Test RMSE:')
test_rmse = accuracy.rmse(svdtuned_test_pred)

In [ ]:
%%time

data.raw_ratings = all_ratings
svdtuned.fit(data.build_full_trainset())

In [ ]:
def get_recommendation(user_id, n=5):
    '''Give n recommendation to user_id'''

    all_books, user_books =  book_read(user_id)
    next_books = [book for book in all_books if book not in user_books]

    if n <= len(next_books):
        ratings = []
        for book in next_books:
            est = svdtuned.predict(user_id, book).est
            ratings.append((book, est))
        ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
        book_ids = [id for id, rate in ratings[:n]]
        return Books[Books.ISBN.isin(book_ids)][['ISBN', 'title', 'author', 'average_rating', 'count_ratings']]
    else:
        print('Please reduce your recommendation request')
        return

In [ ]:
get_recommendation(276729, 5)

In [ ]:
# uncomment this code to dump the calculation result for future use
dump.dump('./dump_tuning', svdtuned_test_pred, svdtuned)

In [ ]:
# uncomment this code to load the dump file
svd_test_pred, svd = dump.load('./dump_svd')
svdtuned_test_pred, svdtuned = dump.load('./dump_tuning')

In [ ]:
df_svd = pd.DataFrame(svd_test_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])
df_svdtuned = pd.DataFrame(svdtuned_test_pred, columns=['uid', 'iid', 'rui', 'est', 'details'])

In [ ]:
from surprise import accuracy

# Hitung RMSE untuk model SVD dasar
print("RMSE for base SVD model:")
accuracy.rmse(svd_test_pred)

# Hitung RMSE untuk model SVD yang di-tuning
print("RMSE for tuned SVD model:")
accuracy.rmse(svdtuned_test_pred)


In [ ]:
#mengali hasil setiap nilai tersebut
df_svd['est'] = df_svd['est'] * 2
df_svdtuned['est'] = df_svdtuned['est'] * 2

fig, ax = plt.subplots(figsize=(8,6))
sns.kdeplot(df_svd['est'], shade=False, color='#92BA92', label='SVD', ax=ax)
sns.kdeplot(df_svdtuned['est'], shade=False, color='#DEBA9D', label='Tuned SVD', ax=ax)
ax.legend()
plt.xlabel('Rating prediction')
plt.ylabel('Density')
plt.title('Comparison of rating estimation in SVD and Tuned SVD')
plt.show()